# Ch13. 数据库

## 13.1 API

标准数据库 API：PEP 249 中定义，也可参考[数据库编程指南](http://wiki.python.org/moin/DatabaseProgramming)

### 13.1.1 全局变量

-   apilevel：字符串常量，用于描述使用的 Python DB API 的版本，
-   threadsafety：0~3 的整数，描述模块的线程安全程度。
    -   0 表示线程不能共享模块
    -   1 表示线程可以共享模块本身，但是不能共享连接
    -   2 表示线程可以共享模块，也可以共享连接，但是不能共享游标
    -   3 表示线程可以完成共享
-   paramstyle：在 SQL 查询中使用的参数风格，即当你执行多个类似的数据库查询时，如何在 SQL 查询中插入参数。
format：表示标准字符串格式设置方式(使用基本的格式编码)，在需要插入参数的地方使用 `%s`
pyformat：表示扩展的格式编码，即旧式字典插入使用的格式编码，例如：`%(foo)s`
qmark：表示使用问号(?)
numeric：表示使用`:1`和`:2`这样的形式表示字段(其中的数字是参数的编号)
named：表示使用`:foobar`这样的形式表示字段(其中`foobar`为参数名称)

### 13.1.2 异常

表13-2：Python DB API 指定的异常
-   StandardError：所有异常的超类
-   Warning(StandardError)：发生非致命问题
-   Error(StandardError)：所有错误条件的超类
-   InterfaceError(Error)：与接口(而不是数据库)相关的错误
-   DatabaseError(Error)：与数据库相关的错误
-   DataError(DatabaseError)：与数据相关的错误，例如：值不在合法的范围内
-   OperationalError(DatabaseError)：数据库操作内部的错误
-   IntegrityError(DatabaseError)：关系完整性的错误，例如：键未通过检查
-   InternalError(DatabaseError)：数据库内部的错误，例如：游标无效
-   ProgrammingError(DatabaseError)：用户编程错误，例如：未找到数据库表
-   NotSupportedError(DatabaseError)：请求不支持的功能，例如：回滚

### 13.1.3 连接

表13-3：函数 connect 的常用参数
-   dsn：数据源的名称，具体含义由数据库确定
-   user：用户名
-   password：密码
-   host：主机名
-   database：数据库名

表13-4：连接对象的方法
-   `close()`：关闭连接对象。
-   `commit()`：提交事务。如果所有事务都已经提交，则返回。
-   `rollback()`：回滚还未提交的事务
-   `cursor()`：返回连接的游标对象

### 13.1.3 游标

表13-5：游标对象的方法
-   `callproc(name[, params])`：使用指定的参数调用指定的数据库过程
-   `close()`：关闭游标
-   `execute(open[, params])`：执行一个 SQL 操作——可能指定参数
-   `executeman(oper, pseq)`：执行指定的 SQL 操作多次，每次都在序列中提供了一组参数
-   `fetchone()`：以序列的方式取回查询结果中的下一行；如果没有更多的行，就返回 None
-   `fetchmany([size])`：取回查询结果中的多行，其中参数 size 的值默认为 arraysize
-   `fetchall()`：以序列的方式取回余下的所有行
-   `nextset()`：跑到下一个结果集
-   `setinputsizes(sizes)`：用于为参数预定义内存区域
-   `setoutputsize(size[, col])`：为取回大量数据而设置缓冲区长度


表13-6：游标对象的属性
-   description：由结果列描述组成的序列(只读)
-   rowcount：结果包含的行数(只读)
-   arraysize：fetchmany 返回的行数，默认为 1

### 13.1.4 类型

表 13-7：DB API 构造函数和特殊值
-   Date(year, month, day)：创建包含日期值的对象
-   Time(hour, minute, second)：创建包含时间值的对象
-   Timestamp(y, mon, d, h, min, s)：创建包含时间戳的对象
-   DataFromTicks(ticks)：根据从新纪元开始过去的秒数创建包含日期值的对象
-   TimeFromTicks(ticks)：：根据从新纪元开始过去的秒数创建包含时间值的对象
-   imestampFromTicks(ticks)：：根据从新纪元开始过去的秒数创建包含时间戳的对象
-   Binary(string)：创建包含二进制字符串的对象
-   STRING：描述基于字符串的列(如：CHAR)
-   BINARY：描述二进制列(如：LONG 或 RAW)
-   NUMBER：描述数字列
-   DATATIME：描述日期/时间列
-   ROWID：描述行 ID 列

## 13.2 SQLite和PySQLite



In [2]:
import sqlite3
conn=sqlite3.connect('somedatabase.db')
curs=conn.cursor()
conn.commit()
conn.close()

### 13.2.2 例子



In [1]:
# 创建并且填充数据库表
# 13-1：将数据导入数据库
import sqlite3

def convert(value):
    if value.startswith('~'): return value.strip('~')
    if not value: value='0'
    return float(value)

conn=sqlite3.connect('food.db')
curs=conn.cursor()
curs.execute('''
CREATE TABLE food(
    id      TEXT PRIMARY KEY,
    desc    TEXT,
    water   FLOAT,
    kcal    FLOAT,
    protein FLOAT,
    fat     FLOAT,
    ash     FLOAT,
    carbs   FLOAT,
    fiber   FLOAT,
    sugar   FLOAT
)
''')
query='INSERT INTO food VALUES(?,?,?,?,?,?,?,?,?,?)'
field_count=10
for line in open('ABBREV.txt'):
    fields=line.split('^')
    vals=[convert(f) for f in fields[:field_count]]
    curs.execute(query,vals)
conn.commit()
conn.close()

In [5]:
import sqlite3,sys

conn=sqlite3.connect('food.db')
curs=conn.cursor()
query='SELECT * FROM food WHERE kcal <=100 AND fiber >=10 ORDER BY sugar'
print(query)
curs.execute(query)
names=[f[0] for f in curs.description]
for row in curs.fetchall():
    for pair in zip(names, row):
        print("{}: {}".format(*pair))
    print()

SELECT * FROM food WHERE kcal <=100 AND fiber >=10 ORDER BY sugar
id: 09216
desc: ORANGE PEEL,RAW
water: 72.5
kcal: 97.0
protein: 1.5
fat: 0.2
ash: 0.8
carbs: 25.0
fiber: 10.6
sugar: 0.0

id: 09156
desc: LEMON PEEL,RAW
water: 81.6
kcal: 47.0
protein: 1.5
fat: 0.3
ash: 0.6
carbs: 16.0
fiber: 10.6
sugar: 4.17

id: 11974
desc: GRAPE LEAVES,RAW
water: 73.32
kcal: 93.0
protein: 5.6
fat: 2.12
ash: 1.65
carbs: 17.31
fiber: 11.0
sugar: 6.3

id: 09231
desc: PASSION-FRUIT,(GRANADILLA),PURPLE,RAW
water: 72.93
kcal: 97.0
protein: 2.2
fat: 0.7
ash: 0.8
carbs: 23.38
fiber: 10.4
sugar: 11.2



## 13.3 小结

-   Python DB API：API 定义了一个简单的标准化接口，所有数据库包装器模块都必须遵循它
-   连接：表示到 SQL 数据库的通信链路，使用方法 cursor 可以从连接获得游标。还可以使用连接对象来提交或者回滚事务。使用完数据库后，注意将连接关闭。
-   游标：用于查询和查看结果。可以逐行取回查询结果，也可以一次取回许多行
-   类型和特殊值：DB API 指定了一组构造函数和特殊值的名称。构造函数用于处理日期和时间对象、二进制对象；特殊值用于表示关系型数据库的类型
-   SQLite：小型的嵌入式 SQL 数据库